# Pygromos: OOP in python 3
__ This Jupyter Notebook is giving you a codification of a Talk I gave in the IGC Seminar __

* Underlying Idea PyGromos:
    * Started during writing the RE-EDS pipeline
    * Try to making all Gromos operations possible from python
    * Sometimes implements the functionally, sometimes hides it

* Goals:
    * Make code easy to read
    * Provide easy access to gromos functions and file modifications (especially in IDEs)
    * Faster adding new features to the code base
    * Add safety functionality/errors to functions

	     increase shareability, reusage of code

In this talk I want to introduce you to Pygromos and look at some aspects of OOP in python3.
Pleas keep in mind that Pygromos right now is an idea not a perfect package, it's under development.

In [1]:
import os, sys
root_dir = os.getcwd()
#if package is not installed and path not set correct - this helps you out :)
sys.path.append(root_dir+"/..")

## Implementations of OOP in Python3

### Definition of a Class
This is the basic definition of a class in python3

#### Magic functions (__<name>__)
    Are functions having two underscores after and before their name.
    These functions do stuff "under the hood" like for example if you interact with an operator.
    Similar to overloadings in C++/Java.
    e.g.: __init__, __str__, __append__, etc.
    More: https://realpython.com/operator-function-overloading/

In [2]:
class treasure_island():
    ## Constructor
    def __init__(self, n_treasures:int):
        self.n_treasures = n_treasures

    ## Other magic function
    def __str__(self)->str:
        return "nice Island with "+str(self.n_treasures)+" treasures"

#MAIN
islandA = treasure_island(n_treasures=2)
print(str(islandA))

nice Island with 2 treasures


### Class Attributes
#### Accessibility
There is no such thing as strict accessiblitiy in python OOP.
The only thing comming close to it, are the naming conventions and name mangling.

In [3]:
class treasure_island():
    fun:int = 1 #Class Variable ~ static
    fun_annotation:str # just an annotation (for example for IDE)

    ## Constructor
    def __init__(self, n_treasures:int, secret_treasure:str="beneath the palm"):
        self.n_treasures = n_treasures
        self._secret_treasure = secret_treasure
        self.__really_secret_treasure = "in the rum barrel"

#MAIN
islandA = treasure_island(2)
print(islandA.n_treasures)
print("Treasure 1 Location: ", islandA._secret_treasure)
#print("Treasure 2 Location: ",islandA.__really_secret_treasure) #throws: treasure_island' object has no attribute '__really_secret_treasure'
print("Treasure 2 Location: ",islandA._treasure_island__really_secret_treasure)

2
Treasure 1 Location:  beneath the palm
Treasure 2 Location:  in the rum barrel


##### Properties
Class Properties are generated via Decorators "@property".
They allow to implement getters and setters in python OOP and even allow preventing setting a variable directly if a setter is missing.

In [4]:
class treasure_island():
    ## Constructor
    def __init__(self, n_treasures:int, secret_treasure:str="beneath the palm"):
        self.n_treasures = n_treasures
        self._secret_treasure = secret_treasure

    @property           #getter
    def treasure(self)->str:
        self.n_treasures -= 1
        return self._secret_treasure

    @treasure.setter    #setter
    def treasure(self, treasure_location:str):
        self.n_treasures += 1
        self._secret_treasure = treasure_location

#MAIN
islandA = treasure_island(1)
print(islandA.treasure, islandA.n_treasures)
islandA.treasure = "under stone"    #Throws an error if @treasure.setter is missing!
print(islandA.n_treasures,islandA.treasure)

beneath the palm 0
1 under stone


#### Mutability of Class Attributes
A very pythonic way of coding is that attributes can be set on the fly(runtime) of a class.
This is nice in small approaches, but be careful with this feature, it's dangerous!

key functionality:
* setattr(obj, key, value)
* hasattr(obj, key) -> bool
* getattr(obj, key) -> value

this is also possibile with functions!

In [5]:
class cnf():
    def __init__(self, content_dict):
        for key, value in content_dict.items():
            setattr(self, key, value)

    def __str__(self)->str:
        #Here you see a list comprehension - that is a functional element.
        return "\n".join([str(key)+"\t"+str(getattr(self, key)) for key in vars(self)])

#MAIN
content_dict = {"POSITION": [1,2,3], "VELOCITY":[4,5,6]}
c = cnf(content_dict)


print("Manual:\n is there a POSITION attribute?\n\t", hasattr(c, "POSITION"), c.POSITION)
print(str(c))


Manual:
 is there a POSITION attribute?
	 True [1, 2, 3]
POSITION	[1, 2, 3]
VELOCITY	[4, 5, 6]


### Methods
#### Levels
there are three types of methods to my knowledge in python:
* instance methods
* class methods
* static methods

In [6]:
class counter():
    class_counter: int = 1

    def __init__(self):
        self.counter: int = 2

    def add_1(self):
        self.counter += 1

    @classmethod
    def class_add_1(cls):
        cls.class_counter += 1
    @staticmethod
    def static_add_1(arg):
        return arg + 1

#MAIN
print(counter.static_add_1(2))

# classmethods
print("Class Var: ", counter.class_counter)
counter.class_add_1()
print("Class Var: ", counter.class_counter)

# normal method:
c = counter()
c.add_1()
print("instance ", c.counter)

3
Class Var:  1
Class Var:  2
instance  3


#### Overwriting methods on Runtime
In python one can overwrite methods of a class with new function definitions during runtime!

In [7]:
class counter():
    def __init__(self):
        self.counter: int = 2
                
    def add_1(self):
        self.counter += 1

    def add_1_modulo(self):
        self.counter = (self.counter + 1) % 2

#MAIN
c = counter()
c.add_1()
print("Normal", c.counter)
c.add_1 = c.add_1_modulo
c.add_1()
print("\"Normal\"", c.counter)

Normal 3
"Normal" 0


### Inheritance
This is a simple inheritance example. There are no multi-inheritances possible (like in Java, not like in C++).
The super constructor can be called any time in the __init__ functions (recommendation: call IT!).

In [8]:
class islands():
    island_name:str
    def __init__(self, island_name):
        self.island_name = island_name

class treasure_island(islands):
    ## Constructor
    def __init__(self, n_treasures:int,
	     secret_treasure:str="beneath the palm"):
        self.n_treasures = n_treasures
        self._secret_treasure = secret_treasure
        super().__init__(island_name="treasure island")

islandA = treasure_island(1)
print(islandA.island_name, islandA.n_treasures)

treasure island 1


#### Type Testing & implicit Polymorphism
as python is not type save, polymorphism is implicitly given.
To ensure that you really encouter a certain class on runtime, one can use the isinstance or issubclass functions.

In [9]:
class islands():
    island_name:str
    def __init__(self, island_name):
        self.island_name = island_name

class treasure_island(islands):
    ## Constructor
    def __init__(self, n_treasures:int,
	     secret_treasure:str="beneath the palm"):
        self.n_treasures = n_treasures
        self._secret_treasure = secret_treasure
        super().__init__(island_name="treasure island")

islandA = treasure_island(1)
boringIsland = islands("nothing here")

#Check
print("the obj boringIsland ..."
      "\n is a treasure_island:",  isinstance(boringIsland, treasure_island),
      "\n is an island:",isinstance(boringIsland, islands),
      "\n is an int:", isinstance(boringIsland, int))
print("the class of boringIsland ..."
      "\n is a subclass of treasure_island:",  issubclass(type(boringIsland), treasure_island),
      "\n is an island:",issubclass(type(boringIsland), islands),
      "\n is an int:", issubclass(type(boringIsland), int))
print()
print("the obj islandA ..."
      "\n is a treasure_island:",  isinstance(islandA, treasure_island),
      "\n is an island:",isinstance(islandA, islands),
      "\n is an int:", isinstance(islandA, int))
print("the class of islandA ..."
      "\n is a subclass of treasure_island:",  issubclass(type(islandA), treasure_island),
      "\n is an island:",issubclass(type(islandA), islands),
      "\n is an int:", issubclass(type(islandA), int))


the obj boringIsland ...
 is a treasure_island: False 
 is an island: True 
 is an int: False
the class of boringIsland ...
 is a subclass of treasure_island: False 
 is an island: True 
 is an int: False

the obj islandA ...
 is a treasure_island: True 
 is an island: True 
 is an int: False
the class of islandA ...
 is a subclass of treasure_island: True 
 is an island: True 
 is an int: False


## Using Pygromos
### PyGromos Wrapper:
#### Example gromosPP:

In [10]:
from pygromos.utils import bash
from pygromos.gromos.gromosPP import GromosPP
from pygromos.data.ff import Gromos54A7

gromPP = GromosPP()
## Make Peptide
sequence = "NH3+ VAL TYR ARG LYSH GLN COO-"
solvent = "H2O"
out_dir =  bash.make_folder(os.path.dirname("example_files/topo"))
out_top_peptide = out_dir+"/peptide.top"


gromPP.make_top(in_building_block_lib_path=Gromos54A7.mtb,
	            in_parameter_lib_path=Gromos54A7.ifp,
                in_sequence=sequence,
                out_top_path= out_top_peptide)


'example_files/peptide.top'

#### Example for gromosXX:

In [ ]:
from pygromos.gromos.gromosXX import GromosXX

gromXX = GromosXX()

# file paths
imd_emin_vacuum = "myPath.imd"
cnf_hpeptide = "myCnf.cnf"
top_vacuum_system = "myTop.top"
out_prefix = "emin_out"
#run emin
gromXX.md_run(in_coord_path=cnf_hpeptide,
              in_topo_path=top_vacuum_system,
              in_imd_path=imd_emin_vacuum,
              out_prefix=out_prefix)

## PyGromos Files:

In [20]:
from pygromos.files.simulation_parameters import imd
from pygromos.files.coord import Cnf
from pygromos.data.simulation_parameters_templates import template_emin
from pygromos.data.solvent_coordinates import spc


cnf_file = Cnf(spc)
residues = cnf_file.get_residues()
lig_num =2


dict_keys(['SOLV'])

In [21]:
#IMD Handling
emin_imd = imd.Imd(template_emin)
emin_imd.SYSTEM.NSM = int(len(residues["SOLV"]))
emin_imd.FORCE.adapt_energy_groups(residues)
emin_imd.edit_EDS(NUMSTATES=len(residues)-1, S = 1.0, EIR=[0.0 for x in range(lig_num)])
emin_imd.write(template_emin)


Setting new EDS_block


'/home/bschroed/Documents/code/PyGromosTools/pygromos/data/simulation_parameters_templates/emin.imd'

## Additional: functional programming - BONUS
Functional programming is yet another programming paradigm. In this style you try not to think in objects, but everything in functions.
A pure functional program is not containing any stored values!

### Lambda functions
* are one line functions, some kind of lazy function definition
* Note the functions are written into variables, which behave like "normal" variables and can be overwritten.
This can happen with any function in python.

In [23]:
# a function
add = lambda x,y: x+y
print("add: ", add(1,2))

# another function
add_modulo = lambda x,y: (x+y)%2
print("add modulo 2: ", add_modulo(1,2))

#override first function (also possible in classes! see next example)
add = add_modulo
print("the new add: ", add(1,2))

add:  3
add modulo 2:  1
the new add:  1


### preimplemented functions
There are the standard functional functions pre implemented in gromos.
* MAP
* ZIP
* FILTER

In [24]:
# MAP
int_list = [1,2,3,4,5]
new_list = list(map(lambda x: x+1, int_list))
print("Map: Add to all elements of the list +1\n ", int_list, "\n ", new_list,"\n")

# ZIP
int_list = [1,2,3,4,5]
letter_list = ["a", "b", "c", "d", "e"]
combination = list(zip(letter_list, int_list))
print("Zip: combined lists: \n ", combination,"\n")

# FILTER
int_list = [1,2,3,4,5]
odd_list = list(filter(lambda x: x%2!=0, int_list))
print("Filter: a list with only odd numbers: \n ", odd_list,"\n")

Map: Add to all elements of the list +1
  [1, 2, 3, 4, 5] 
  [2, 3, 4, 5, 6] 

Zip: combined lists: 
  [('a', 1), ('b', 2), ('c', 3), ('d', 4), ('e', 5)] 

Filter: a list with only odd numbers: 
  [1, 3, 5] 



### list/dict - comprehensions
can be used to bring a for loop into one line.

In [25]:
# list comprehension
int_list = [1,2,3,4,5]
odd_list = [x for x in int_list if(x%2!=0)]
odd_replace_list = [x if(x%2!=0) else 0 for x in int_list]

print("list comprehension")
print("exclude odds: ", odd_list)
print("replace odds: ", odd_replace_list)
print()

#dict comprehension
int_list = [1,2,3,4,5]
letter_list = ["a", "b", "c", "d", "e"]

combination_dict = {key: value for key, value in zip(letter_list, int_list)}
combination_list = list(combination_dict.items()) #if it needs to be a list
print("dict comprehension")
print("combinatinos dict: ", combination_dict)
print("combinatinos list: ", combination_list)
print()

list comprehension
exclude odds:  [1, 3, 5]
replace odds:  [1, 0, 3, 0, 5]

dict comprehension
combinatinos dict:  {'a': 1, 'b': 2, 'c': 3, 'd': 4, 'e': 5}
combinatinos list:  [('a', 1), ('b', 2), ('c', 3), ('d', 4), ('e', 5)]



### decorators - how they work
Decorators are actually function wrapping a function, that builds a new function! :)
This is part of higher order functional programming and therefore for the pros.

In [26]:
#Our nice Decorator
def secret(function:callable):
    def disclosure(): #the "final" function
        print("don't tell about the island\n")
        function()  #the decorated function
        print("\nDamn!\n")
    return disclosure

#Our Application
@secret
def treasure_island_location():
    print("\tnice island, with two mountains and train traffic")


treasure_island_location()


don't tell about the island

	nice island, with two mountains and train traffic

Damn!



I hope you liked the talk.
Thanks to:

 Sereina, Phil

 Salome, Marc, Candide

 and all the IGC Group Members!